In [1]:
from enum import Enum
from cuery import Prompt, ResponseModel, Task
from pydantic import Field

# Choices

In [ ]:
class Role(Enum):
    PRINCIPAL = "PRINCIPAL"
    TEACHER = "TEACHER"
    STUDENT = "STUDENT"
    OTHER = "OTHER"


class UserDetail(ResponseModel):
    age: int
    name: str
    role: Role = Field(description="Correctly assign one of the predefined roles to the user.")


prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "Please a create a synthetic user profile with age, name and role.",
        }
    ]
)

task = Task("user_profile", prompt=prompt, response=UserDetail)

response = await task()
print(response)
response.to_pandas()

[UserDetail(age=25, name='Emily', role=<Role.STUDENT: 'STUDENT'>)]


,age,name,role
0,25,Emily,Role.STUDENT


# Simple Multivalued fields

In [21]:
class Ingredients(ResponseModel):
    items: list[str] = Field(description="List of ingredients for the recipe.")


prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "List the ingredients for the following dish: {{dish}}.",
        }
    ],
    required=["dish"],
)

context = [{"dish": "pasta bolognese"}, {"dish": "chocolate cake"}]
task = Task("dishes", prompt=prompt, response=Ingredients)
responses = await task(context=context)
print(responses)

Iterating context:   0%|          | 0/2 [00:00<?, ?it/s]

[Ingredients(items=['pasta', 'minced meat', 'onion', 'garlic', 'tomato sauce', 'Italian seasoning', 'salt', 'pepper', 'olive oil', 'parmesan cheese']), Ingredients(items=['chocolate', 'flour', 'sugar', 'eggs', 'butter', 'cocoa powder', 'baking powder', 'vanilla extract', 'salt', 'milk'])]


In [22]:
responses.to_pandas(explode=False)

,dish,items
0,pasta bolognese,"[pasta, minced meat, onion, garlic, tomato sau..."
1,chocolate cake,"[chocolate, flour, sugar, eggs, butter, cocoa ..."


In [23]:
responses.to_records(explode=False)

[{'dish': 'pasta bolognese',
  'items': ['pasta',
   'minced meat',
   'onion',
   'garlic',
   'tomato sauce',
   'Italian seasoning',
   'salt',
   'pepper',
   'olive oil',
   'parmesan cheese']},
 {'dish': 'chocolate cake',
  'items': ['chocolate',
   'flour',
   'sugar',
   'eggs',
   'butter',
   'cocoa powder',
   'baking powder',
   'vanilla extract',
   'salt',
   'milk']}]

In [ ]:
import importlib
from cuery import response

importlib.reload(response)

r = response.ResponseSet(responses.responses, context=context, required=prompt.required)
r.to_records()

[{'dish': 'pasta bolognese', 'items': 'pasta'},
 {'dish': 'pasta bolognese', 'items': 'minced meat'},
 {'dish': 'pasta bolognese', 'items': 'onion'},
 {'dish': 'pasta bolognese', 'items': 'garlic'},
 {'dish': 'pasta bolognese', 'items': 'tomato sauce'},
 {'dish': 'pasta bolognese', 'items': 'Italian seasoning'},
 {'dish': 'pasta bolognese', 'items': 'salt'},
 {'dish': 'pasta bolognese', 'items': 'pepper'},
 {'dish': 'pasta bolognese', 'items': 'olive oil'},
 {'dish': 'pasta bolognese', 'items': 'parmesan cheese'},
 {'dish': 'chocolate cake', 'items': 'chocolate'},
 {'dish': 'chocolate cake', 'items': 'flour'},
 {'dish': 'chocolate cake', 'items': 'sugar'},
 {'dish': 'chocolate cake', 'items': 'eggs'},
 {'dish': 'chocolate cake', 'items': 'butter'},
 {'dish': 'chocolate cake', 'items': 'cocoa powder'},
 {'dish': 'chocolate cake', 'items': 'baking powder'},
 {'dish': 'chocolate cake', 'items': 'vanilla extract'},
 {'dish': 'chocolate cake', 'items': 'salt'},
 {'dish': 'chocolate cake', '